In [44]:
import torch
import torch.nn as nn
import random
import pandas as pd
from tqdm import tqdm

import numpy as np
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from pathlib import Path

In [45]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', DEVICE)

data_path= '/kaggle/input/aml-competition/train/train/train.npz'
test_path= '/kaggle/input/aml-competition/test/test/test.clean.npz'

data = np.load(Path('/kaggle/input/aml-competition/train/train/train.npz'))

caption_embeddings = data['captions/embeddings']
image_embeddings = data['images/embeddings']
caption_labels = data['captions/label']

data.close()

x_abs = torch.from_numpy(caption_embeddings) # captions space
y_abs = torch.from_numpy(image_embeddings[np.argmax(caption_labels, axis=1)]) # images space

print('Captions latent space:', x_abs.shape)
print('Images latent space:', y_abs.shape)

Device: cuda
Captions latent space: torch.Size([125000, 1024])
Images latent space: torch.Size([125000, 1536])


In [46]:
class LinearTranslator(nn.Module):
    def __init__(self, dim: int, use_bias: bool, xavier_init: bool):
        super().__init__()
        assert isinstance(dim, int) and dim > 0, "Expecting positive dimension"
        assert isinstance(use_bias, bool), 'Expecting boolean param for "use_bias"'
        assert isinstance(xavier_init, bool), 'Expecting boolean param for "xavier_init"'
                
        self.linear = nn.Linear(dim, dim, bias=use_bias)

        if xavier_init:
            self._xavier_init()
    
    def _xavier_init(self):
            nn.init.xavier_uniform_(self.linear.weight)
            
            if self.linear.bias is not None:
                nn.init.constant_(self.linear.bias, 0)
    
    def forward(self, x):
        return self.linear(x)


class MLPTranslator(nn.Module):
    def __init__(self, input_dim, hidden_dims=[1200, 600, 300], output_dim=None, dropout=0.15):
        super().__init__()
        if output_dim is None:
            output_dim = input_dim

        layers = []
        in_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(in_dim, h_dim))
            layers.append(nn.LayerNorm(h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_dim = h_dim
        layers.append(nn.Linear(in_dim, output_dim))  # output layer

        self.net = nn.Sequential(*layers)

        # Xavier initialization
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)



def train_translator(model: LinearTranslator, train_loader: DataLoader, val_loader: DataLoader, epochs: int, lr: float):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()

        train_loss = 0
        for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)

            optimizer.zero_grad()

            # outputs = model(X_batch)

            y_batch = F.normalize(y_batch, p=2, dim=1)
            outputs = F.normalize(model(X_batch), p=2, dim=1)
            
            loss = 1 - (outputs * y_batch).sum(dim=1).mean()

            loss.backward()

            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()

        val_loss = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                
                y_batch = F.normalize(y_batch, p=2, dim=1)
                outputs = F.normalize(model(X_batch), p=2, dim=1)
                
                loss = 1 - (outputs * y_batch).sum(dim=1).mean()

                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}")

    return model

In [47]:
def center(data: torch.Tensor):
    return data - data.mean(dim=0, keepdim=True)


def normalize(data: torch.Tensor):
    return F.normalize(data, p=2, dim=1)


def extract_anchors(x_abs: torch.Tensor, y_abs: torch.Tensor, anchors_number: int):
    indices = torch.randperm(x_abs.size(0))[:anchors_number]
    
    return x_abs[indices], y_abs[indices]


def releative_representation(data: torch.Tensor, anchors: torch.Tensor):
    return F.normalize(data, p=2, dim=1) @ F.normalize(anchors.T, p=2, dim=1)

In [48]:
# omega = 8
# delta = 0.65
anchors_number = 1000

x = normalize(center(x_abs))
y = normalize(center(y_abs))

x_anchors, y_anchors = extract_anchors(x, y, anchors_number)

x_rel = x @ x_anchors.T # releative_representation(x_centered, x_anchors)
y_rel = y @ y_anchors.T # releative_representation(y_centered, y_anchors)

In [49]:
n_train = int(0.9 * x_rel.shape[0])
batch_size = 256

indices = torch.randperm(x_rel.shape[0])
train_idx = indices[:n_train]
val_idx = indices[n_train:]

x_train, x_val = x_rel[train_idx], x_rel[val_idx]
y_train, y_val = y_rel[train_idx], y_rel[val_idx]

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [50]:
epochs = 300
lr = 0.0003

#model = LinearTranslator(x_rel.shape[1], True, True).to(DEVICE)
model = MLPTranslator(x_rel.shape[1]).to(DEVICE)

train_translator(model, train_loader, val_loader, epochs, lr)

Epoch 1/300: 100%|██████████| 440/440 [00:02<00:00, 169.63it/s]


Epoch 1: Train Loss = 0.425923, Val Loss = 0.342918


Epoch 2/300: 100%|██████████| 440/440 [00:02<00:00, 176.59it/s]


Epoch 2: Train Loss = 0.343787, Val Loss = 0.324698


Epoch 3/300: 100%|██████████| 440/440 [00:06<00:00, 66.78it/s] 


...

Epoch 299/300: 100%|██████████| 440/440 [00:02<00:00, 172.75it/s]


Epoch 299: Train Loss = 0.172019, Val Loss = 0.275631


Epoch 300/300: 100%|██████████| 440/440 [00:02<00:00, 176.17it/s]


Epoch 300: Train Loss = 0.171668, Val Loss = 0.276042


MLPTranslator(
  (net): Sequential(
    (0): Linear(in_features=1000, out_features=1200, bias=True)
    (1): LayerNorm((1200,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.15, inplace=False)
    (4): Linear(in_features=1200, out_features=600, bias=True)
    (5): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
    (6): ReLU()
    (7): Dropout(p=0.15, inplace=False)
    (8): Linear(in_features=600, out_features=300, bias=True)
    (9): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (10): ReLU()
    (11): Dropout(p=0.15, inplace=False)
    (12): Linear(in_features=300, out_features=1000, bias=True)
  )
)

In [51]:
torch.save({
    'model_state_dict': model.state_dict(),
    'x_anchors': x_anchors,
    'y_anchors': y_anchors
}, 'translator_with_anchors.pth')

In [52]:
with torch.no_grad():
    y_rel_pred = model(x_rel.to(DEVICE)).cpu()  # output del modello nello spazio relativo immagini

# Cosine similarity tra y_rel e y_rel_pred
cos_sim = F.cosine_similarity(y_rel_pred, y_rel, dim=1)
print("Media cosine similarity nello spazio relativo:", cos_sim.mean().item())
print("Cosine similarity min/max:", cos_sim.min().item(), cos_sim.max().item())

Media cosine similarity nello spazio relativo: 0.8538557291030884
Cosine similarity min/max: -0.709624707698822 0.9906015992164612


In [53]:
pseudo_inverse = torch.linalg.pinv(y_anchors.T).to(DEVICE)


def generate_submission(model, test_path: Path, output_file="submission.csv", device=None):
    test_data = np.load(test_path)
    sample_ids = test_data['captions/ids']
    test_embds = test_data['captions/embeddings']
    test_data.close()
    
    test_embds = torch.from_numpy(test_embds).float()
    test_embds = normalize(center(test_embds)) # like training

    with torch.no_grad():
        x_rel_test = (test_embds @ x_anchors.T).to(DEVICE)
        pred_embds = model(x_rel_test) @ pseudo_inverse

    print('Y abs reconstructed shape:', pred_embds.shape)

    print("Generating submission file...")

    if isinstance(pred_embds, torch.Tensor):
        pred_embds = pred_embds.cpu().numpy()

    df_submission = pd.DataFrame({'id': sample_ids, 'embedding': pred_embds.tolist()})

    df_submission.to_csv(output_file, index=False, float_format='%.17g')
    print(f"✓ Saved submission to {output_file}")

    return df_submission


generate_submission(model, Path(test_path), device=DEVICE)

Y abs reconstructed shape: torch.Size([1500, 1536])
Generating submission file...
✓ Saved submission to submission.csv


,id,embedding
0,1,"[0.04971304163336754, 0.003106743097305298, -0..."
1,2,"[-0.022800683975219727, -0.03812825679779053, ..."
2,3,"[-0.007564276456832886, -0.053823813796043396,..."
3,4,"[-0.0014978647232055664, 0.04147624969482422, ..."
4,5,"[0.04639643058180809, -0.0068305134773254395, ..."
...,...,...
1495,1496,"[0.006414793431758881, -0.023131832480430603, ..."
1496,1497,"[0.0017552375793457031, 0.07606494426727295, -..."
1497,1498,"[0.012645125389099121, 0.0034373849630355835, ..."
1498,1499,"[0.02855837345123291, 0.0019985437393188477, -..."
